In [16]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
print("Current environment:", os.environ.get('CONDA_DEFAULT_ENV', 'None'))
print("LD_LIBRARY_PATH set:", 'LD_LIBRARY_PATH' in os.environ)
print("TensorFlow version:", tf.__version__)
print("GPUs found:", len(tf.config.list_physical_devices('GPU')))

Current environment: tf-clean
LD_LIBRARY_PATH set: True
TensorFlow version: 2.15.0
GPUs found: 1


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

Path to dataset files: /home/francisco_ardoso/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2


In [3]:
import os
import pandas as pd
meta=pd.read_csv(os.path.join(path,"HAM10000_metadata.csv"))



In [4]:
meta.dtypes

lesion_id        object
image_id         object
dx               object
dx_type          object
age             float64
sex              object
localization     object
dtype: object

In [5]:
meta.head()
meta.dropna()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [6]:
meta.age.value_counts()


age
45.0    1299
50.0    1187
55.0    1009
40.0     985
60.0     803
70.0     756
35.0     753
65.0     731
75.0     618
30.0     464
80.0     404
85.0     290
25.0     247
20.0     169
5.0       86
15.0      77
10.0      41
0.0       39
Name: count, dtype: int64

In [7]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import streamlit as st
def eda():
    meta_clean = meta.dropna()

    # Prep grouped counts
    dx_counts = meta_clean["dx"].value_counts().reset_index()
    dx_counts.columns = ["diagnosis", "count"]

    loc_counts = meta_clean["localization"].value_counts().reset_index()
    loc_counts.columns = ["localization", "count"]

    # Create 2x2 grid + 1 full-width bottom row
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            "Age Distribution", 
            "Diagnosis Counts", 
            "Sex Distribution", 
            "Lesion Localization",
            "Age by Diagnosis & Sex", ""
        ),
        specs=[
            [{"type": "xy"}, {"type": "xy"}],
            [{"type": "domain"}, {"type": "xy"}],   # <-- pie goes into domain
            [{"type": "xy", "colspan": 2}, None]
        ]
    )

    # --- 1) Age histogram
    fig.add_trace(
        go.Histogram(
            x=meta_clean["age"], nbinsx=30, opacity=0.7, name="Age",
            marker=dict(color="#1f77b4")
        ),
        row=1, col=1
    )

    # --- 2) Diagnosis bar
    fig.add_trace(
        go.Bar(
            x=dx_counts["diagnosis"], 
            y=dx_counts["count"], 
            name="Diagnosis",
            marker=dict(color=px.colors.qualitative.Dark2)
        ),
        row=1, col=2
    )

    # --- 3) Sex pie
    fig.add_trace(
        go.Pie(
            labels=meta_clean["sex"], 
            hole=0.4,
            name="Sex"
        ),
        row=2, col=1
    )

    # --- 4) Localization bar
    fig.add_trace(
        go.Bar(
            x=loc_counts["localization"], 
            y=loc_counts["count"], 
            name="Localization",
            marker=dict(color=px.colors.qualitative.Pastel)
        ),
        row=2, col=2
    )

    # --- 5) Age vs Diagnosis (scatter instead of strip for compatibility)
    fig.add_trace(
        go.Scatter(
            x=meta_clean["dx"], 
            y=meta_clean["age"], 
            mode="markers",
            marker=dict(size=6, color=meta_clean["sex"].map({"male": "blue", "female": "red"})),
            name="Age by Dx & Sex"
        ),
        row=3, col=1
    )

    # Layout
    fig.update_layout(
        title_text="HAM10000 Dataset Overview",
        height=900,
        showlegend=True,
        template="plotly_white"
    )

    return st.plotly_chart(fig, use_container_width=True)


In [8]:
#transform everything into categorical
meta['label'] = meta['dx'].astype('category').cat.codes



In [9]:


# Attach image file paths as these will be the feature paths
img_dir_1 = os.path.join(path, "HAM10000_images_part_1")
img_dir_2 = os.path.join(path, "HAM10000_images_part_2")

def get_path(image_id):
    p1 = os.path.join(img_dir_1, image_id + ".jpg")
    p2 = os.path.join(img_dir_2, image_id + ".jpg")
    return p1 if os.path.exists(p1) else p2

meta['file_path'] = meta['image_id'].apply(get_path)
meta.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,label,file_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,2,/home/francisco_ardoso/.cache/kagglehub/datase...
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,2,/home/francisco_ardoso/.cache/kagglehub/datase...
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,2,/home/francisco_ardoso/.cache/kagglehub/datase...
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,2,/home/francisco_ardoso/.cache/kagglehub/datase...
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,2,/home/francisco_ardoso/.cache/kagglehub/datase...


In [10]:
df=meta[["file_path","dx","label"]]
num_classes=len(df.label.unique())

In [11]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
df=meta
row = df.iloc[0]

img = mpimg.imread(row['file_path'])

plt.imshow(img)
plt.title(f"Label: {row['label']} (dx: {row['dx']})")
plt.axis('off')
plt.show()

/tmp/ipykernel_2248/833956166.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
X = meta['file_path'].values   # list of file paths
y = meta['label'].values       # numeric labels
img.shape #450 by 600, RGB=3

(450, 600, 3)

In [13]:
# %%
# ============================================================================
# PREPARE DATA FOR BOTH MODELS (CV-only and Multimodal)
# ============================================================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Handle missing values
meta['age'].fillna(meta['age'].median(), inplace=True)
meta['sex'].fillna(meta['sex'].mode()[0], inplace=True)

# Encode categorical variables for tabular features
le_sex = LabelEncoder()
le_loc = LabelEncoder()

meta['sex_encoded'] = le_sex.fit_transform(meta['sex'])
meta['localization_encoded'] = le_loc.fit_transform(meta['localization'])

# Prepare data
tabular_features = ['age', 'sex_encoded', 'localization_encoded']
X_tabular = meta[tabular_features].values
X_images = meta['file_path'].values
y = meta['label'].values

print(f"Tabular features shape: {X_tabular.shape}")
print(f"Images: {len(X_images)}")
print(f"Labels: {len(y)}")

Tabular features shape: (10015, 3)
Images: 10015
Labels: 10015


/tmp/ipykernel_2248/2353864981.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  meta['age'].fillna(meta['age'].median(), inplace=True)
/tmp/ipykernel_2248/2353864981.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [14]:
# Split data for both models
X_img_train, X_img_temp, X_tab_train, X_tab_temp, y_train, y_temp = train_test_split(
    X_images, X_tabular, y, test_size=0.3, stratify=y, random_state=42
)

X_img_val, X_img_test, X_tab_val, X_tab_test, y_val, y_test = train_test_split(
    X_img_temp, X_tab_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Scale tabular features
scaler = StandardScaler()
X_tab_train = scaler.fit_transform(X_tab_train)
X_tab_val = scaler.transform(X_tab_val)
X_tab_test = scaler.transform(X_tab_test)

print(f"\nTrain: {len(X_img_train)}, Val: {len(X_img_val)}, Test: {len(X_img_test)}")


Train: 7010, Val: 1502, Test: 1503


In [17]:
# %%
# Processing functions
IMG_SIZE = 224

def process_image(file_path, label):
    """Process image for CV model"""
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE], method="bicubic")
    img = img / 255.0
    return img, label

def process_multimodal(file_path, tabular, label):
    """Process image + tabular for multimodal model"""
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE], method="bicubic")
    img = img / 255.0
    return {'image': img, 'tabular': tabular}, label

# %%
# Creating datasets for BOTH models
BATCH_SIZE = 32

# CV-only datasets
def make_cv_dataset(X_img, y, training=True):
    ds = tf.data.Dataset.from_tensor_slices((X_img, y))
    ds = ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(1000)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Multimodal datasets
def make_multimodal_dataset(X_img, X_tab, y, training=True):
    ds = tf.data.Dataset.from_tensor_slices((X_img, X_tab.astype(np.float32), y))
    ds = ds.map(process_multimodal, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(1000)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Create CV-only datasets
train_ds_cv = make_cv_dataset(X_img_train, y_train, training=True)
val_ds_cv = make_cv_dataset(X_img_val, y_val, training=False)
test_ds_cv = make_cv_dataset(X_img_test, y_test, training=False)

# Create multimodal datasets
train_ds_mm = make_multimodal_dataset(X_img_train, X_tab_train, y_train, training=True)
val_ds_mm = make_multimodal_dataset(X_img_val, X_tab_val, y_val, training=False)
test_ds_mm = make_multimodal_dataset(X_img_test, X_tab_test, y_test, training=False)

print("✓ Datasets created for both CV-only and Multimodal models")

✓ Datasets created for both CV-only and Multimodal models


In [18]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# %%
print("="*70)
print("MODEL 1: PURE COMPUTER VISION (IMAGE ONLY)")
print("="*70)

tf.keras.backend.clear_session()

base_model_cv = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
base_model_cv.trainable = False

model_cv = models.Sequential([
    base_model_cv,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),  # Add extra layer
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
], name="HAM_ResNet_CV")

model_cv.summary()

# %%
# Compile and train CV model
model_cv.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights_array))

callbacks_cv = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model_cv.keras', save_best_only=True, monitor='val_accuracy')
]

history_cv = model_cv.fit(
    train_ds_cv,  # Changed to train_ds_cv
    validation_data=val_ds_cv,  # Changed
    epochs=100,
    class_weight=class_weights,
    callbacks=callbacks_cv
)

# %%
# Evaluate CV model
print("\nCV Model - Initial Training Results")
test_loss_cv_initial, test_accuracy_cv_initial = model_cv.evaluate(test_ds_cv)
print(f"Test Accuracy: {test_accuracy_cv_initial:.4f}")

# %%
# Fine-tune CV model
base_model_cv.trainable = True

model_cv.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_cv_finetune = model_cv.fit(
    train_ds_cv,
    validation_data=val_ds_cv,
    epochs=20,
    class_weight=class_weights
)

test_loss_cv, test_accuracy_cv = model_cv.evaluate(test_ds_cv)
print(f"CV Test Accuracy (final): {test_accuracy_cv:.4f}")

model_cv.save('HAM_ResNet_CV_final.keras')

MODEL 1: PURE COMPUTER VISION (IMAGE ONLY)
Model: "HAM_ResNet_CV"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
          

2025-10-01 11:12:36.870788: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2025-10-01 11:12:37.079229: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2025-10-01 11:12:37.079303: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2025-10-01 11:12:37.079423: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2025-10-01 11:12:38.923462: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:504] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for 

220/220 [==============================] - 39s 127ms/step - loss: 2.0931 - accuracy: 0.1378 - val_loss: 1.9489 - val_accuracy: 0.1099
Epoch 2/100
220/220 [==============================] - 25s 113ms/step - loss: 1.9591 - accuracy: 0.0991 - val_loss: 1.9457 - val_accuracy: 0.0153
Epoch 3/100
220/220 [==============================] - 25s 113ms/step - loss: 1.9556 - accuracy: 0.1282 - val_loss: 1.9521 - val_accuracy: 0.0113
Epoch 4/100
220/220 [==============================] - 25s 111ms/step - loss: 1.9491 - accuracy: 0.1001 - val_loss: 2.1268 - val_accuracy: 0.0113
Epoch 5/100
220/220 [==============================] - 25s 112ms/step - loss: 1.9525 - accuracy: 0.0254 - val_loss: 2.0188 - val_accuracy: 0.0113
Epoch 6/100
220/220 [==============================] - 23s 103ms/step - loss: 1.9493 - accuracy: 0.0690 - val_loss: 1.9428 - val_accuracy: 0.0140
Epoch 7/100
220/220 [==============================] - 26s 116ms/step - loss: 1.9477 - accuracy: 0.2773 - val_loss: 1.9348 - val_accurac

2025-10-01 11:20:26.285044: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-10-01 11:20:26.350341: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-10-01 11:20:43.172746: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-10-01 11:20:43.260137: W external/local_tsl/tsl/framework/bfc_

220/220 [==============================] - 88s 228ms/step - loss: 1.6129 - accuracy: 0.5424 - val_loss: 1.5639 - val_accuracy: 0.6698
Epoch 2/20
220/220 [==============================] - 53s 238ms/step - loss: 1.0727 - accuracy: 0.6208 - val_loss: 2.7796 - val_accuracy: 0.6691
Epoch 3/20
220/220 [==============================] - 49s 219ms/step - loss: 0.8193 - accuracy: 0.6524 - val_loss: 2.2793 - val_accuracy: 0.0353
Epoch 4/20
220/220 [==============================] - 51s 232ms/step - loss: 0.6030 - accuracy: 0.7185 - val_loss: 1.6063 - val_accuracy: 0.3562
Epoch 5/20
220/220 [==============================] - 49s 219ms/step - loss: 0.4894 - accuracy: 0.7327 - val_loss: 1.2004 - val_accuracy: 0.6578
Epoch 6/20
220/220 [==============================] - 48s 218ms/step - loss: 0.3100 - accuracy: 0.8080 - val_loss: 0.6991 - val_accuracy: 0.7417
Epoch 7/20
220/220 [==============================] - 52s 233ms/step - loss: 0.2908 - accuracy: 0.8328 - val_loss: 0.7192 - val_accuracy: 0.7

In [19]:
# %%
print("\n" + "="*70)
print("MODEL 2: MULTIMODAL (IMAGE + TABULAR)")
print("="*70)

tf.keras.backend.clear_session()

from tensorflow.keras import Input, Model

# Image input branch
image_input = Input(shape=(224, 224, 3), name='image')
base_resnet = ResNet50(include_top=False, weights='imagenet')(image_input)
x = layers.GlobalAveragePooling2D()(base_resnet)
x = layers.Dropout(0.5)(x)
image_features = layers.Dense(128, activation='relu')(x)

# Tabular input branch
tabular_input = Input(shape=(len(tabular_features),), name='tabular')
tab = layers.Dense(64, activation='relu')(tabular_input)
tab = layers.BatchNormalization()(tab)
tab = layers.Dropout(0.3)(tab)
tab = layers.Dense(64, activation='relu')(tab)
tab = layers.BatchNormalization()(tab)
tab = layers.Dense(32, activation='relu')(tab)

# Combine both branches
combined = layers.concatenate([image_features, tab])
combined = layers.Dense(64, activation='relu')(combined)
combined = layers.Dropout(0.3)(combined)
output = layers.Dense(num_classes, activation='softmax')(combined)

# Create model
model_mm = Model(
    inputs=[image_input, tabular_input],
    outputs=output,
    name="HAM_ResNet_Multimodal"
)

# Freeze ResNet base
for layer in model_mm.layers:
    if isinstance(layer, tf.keras.Model):
        layer.trainable = False

model_mm.summary()

# %%
# Compile and train multimodal
model_mm.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_mm = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model_mm.keras', save_best_only=True, monitor='val_accuracy')
]

history_mm = model_mm.fit(
    train_ds_mm,
    validation_data=val_ds_mm,
    epochs=100,
    class_weight=class_weights,
    callbacks=callbacks_mm
)

# %%
# Evaluate multimodal
print("\nMultimodal Model - Initial Training Results")
test_loss_mm_initial, test_accuracy_mm_initial = model_mm.evaluate(test_ds_mm)
print(f"Test Accuracy: {test_accuracy_mm_initial:.4f}")

# %%
# Fine-tune multimodal
for layer in model_mm.layers:
    layer.trainable = True

model_mm.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_mm_finetune = model_mm.fit(
    train_ds_mm,
    validation_data=val_ds_mm,
    epochs=20,
    class_weight=class_weights
)

test_loss_mm, test_accuracy_mm = model_mm.evaluate(test_ds_mm)
print(f"Multimodal Test Accuracy (final): {test_accuracy_mm:.4f}")

model_mm.save('HAM_ResNet_Multimodal_final.keras')

# %%
# COMPARISON
print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(f"CV-Only:    {test_accuracy_cv:.4f} ({test_accuracy_cv*100:.2f}%)")
print(f"Multimodal: {test_accuracy_mm:.4f} ({test_accuracy_mm*100:.2f}%)")
improvement = (test_accuracy_mm - test_accuracy_cv) * 100
print(f"Improvement: {improvement:+.2f}%")




MODEL 2: MULTIMODAL (IMAGE + TABULAR)
Model: "HAM_ResNet_Multimodal"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 tabular (InputLayer)        [(None, 3)]                  0         []                            
                                                                                                  
 dense_1 (Dense)             (None, 64)                   256       ['tabular[0][0]']             
                                                                                                  
 image (InputLayer)          [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 batch_normalization (Batch  (None, 64)                   256       ['dense_1[0][0]']             
 Normalization)                        

2025-10-01 11:43:36.820817: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-01 11:43:36.828066: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-01 11:43:36.831928: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-01 11:43:36.832610: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-01 11:43:36.832751: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-01

220/220 [==============================] - 35s 129ms/step - loss: 1.9364 - accuracy: 0.2392 - val_loss: 1.8424 - val_accuracy: 0.2310
Epoch 2/100
220/220 [==============================] - 26s 117ms/step - loss: 1.8321 - accuracy: 0.2526 - val_loss: 1.7985 - val_accuracy: 0.2477
Epoch 3/100
220/220 [==============================] - 28s 124ms/step - loss: 1.8067 - accuracy: 0.2213 - val_loss: 1.6538 - val_accuracy: 0.3489
Epoch 4/100
220/220 [==============================] - 23s 104ms/step - loss: 1.7622 - accuracy: 0.2392 - val_loss: 1.6697 - val_accuracy: 0.1977
Epoch 5/100
220/220 [==============================] - 26s 115ms/step - loss: 1.7605 - accuracy: 0.2425 - val_loss: 1.6930 - val_accuracy: 0.1485
Epoch 6/100
220/220 [==============================] - 26s 117ms/step - loss: 1.7448 - accuracy: 0.2231 - val_loss: 1.6386 - val_accuracy: 0.1791
Epoch 7/100
220/220 [==============================] - 26s 117ms/step - loss: 1.7518 - accuracy: 0.2190 - val_loss: 1.6309 - val_accurac

In [20]:
# %%
# Save everything for Streamlit
import pickle
import joblib

np.save('y_pred_cv.npy', np.argmax(model_cv.predict(test_ds_cv), axis=1))
np.save('y_pred_mm.npy', np.argmax(model_mm.predict(test_ds_mm), axis=1))
np.save('y_test.npy', y_test)

with open('histories.pkl', 'wb') as f:
    pickle.dump({
        'history_cv': history_cv.history,
        'history_cv_finetune': history_cv_finetune.history,
        'history_mm': history_mm.history,
        'history_mm_finetune': history_mm_finetune.history
    }, f)

joblib.dump(scaler, 'scaler.pkl')
joblib.dump(le_sex, 'le_sex.pkl')
joblib.dump(le_loc, 'le_loc.pkl')

print("\n✓ All files saved!")

47/47 [==============================] - 5s 78ms/step

✓ All files saved!


In [21]:
# %%
# ============================================================================
# VISUALIZATION: COMPARE BOTH MODELS
# ============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# Get predictions for both models
print("Generating predictions for both models...")
y_pred_cv_probs = model_cv.predict(test_ds_cv)
y_pred_cv = np.argmax(y_pred_cv_probs, axis=1)

y_pred_mm_probs = model_mm.predict(test_ds_mm)
y_pred_mm = np.argmax(y_pred_mm_probs, axis=1)

class_names = meta['dx'].astype('category').cat.categories.tolist()

# %%
# 1. TRAINING HISTORY COMPARISON
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Training History Comparison: CV vs Multimodal', fontsize=16, fontweight='bold')

# CV Model
axes[0, 0].plot(history_cv.history['accuracy'], label='CV Train', linewidth=2, color='#1f77b4')
axes[0, 0].plot(history_cv.history['val_accuracy'], label='CV Val', linewidth=2, color='#ff7f0e')
axes[0, 0].set_title('CV Model - Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history_cv.history['loss'], label='CV Train', linewidth=2, color='#1f77b4')
axes[0, 1].plot(history_cv.history['val_loss'], label='CV Val', linewidth=2, color='#ff7f0e')
axes[0, 1].set_title('CV Model - Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Multimodal Model
axes[1, 0].plot(history_mm.history['accuracy'], label='MM Train', linewidth=2, color='#2ca02c')
axes[1, 0].plot(history_mm.history['val_accuracy'], label='MM Val', linewidth=2, color='#d62728')
axes[1, 0].set_title('Multimodal Model - Accuracy')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(history_mm.history['loss'], label='MM Train', linewidth=2, color='#2ca02c')
axes[1, 1].plot(history_mm.history['val_loss'], label='MM Val', linewidth=2, color='#d62728')
axes[1, 1].set_title('Multimodal Model - Loss')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# %%
# 2. SIDE-BY-SIDE CONFUSION MATRICES
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# CV Model Confusion Matrix
cm_cv = confusion_matrix(y_test, y_pred_cv)
sns.heatmap(cm_cv, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names, ax=axes[0])
axes[0].set_title(f'CV Model (Acc: {test_accuracy_cv:.4f})', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')

# Multimodal Confusion Matrix
cm_mm = confusion_matrix(y_test, y_pred_mm)
sns.heatmap(cm_mm, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names, ax=axes[1])
axes[1].set_title(f'Multimodal Model (Acc: {test_accuracy_mm:.4f})', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')

plt.tight_layout()
plt.savefig('confusion_matrices_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# %%
# 3. PER-CLASS F1-SCORE COMPARISON
precision_cv, recall_cv, f1_cv, support_cv = precision_recall_fscore_support(
    y_test, y_pred_cv, labels=range(num_classes)
)

precision_mm, recall_mm, f1_mm, support_mm = precision_recall_fscore_support(
    y_test, y_pred_mm, labels=range(num_classes)
)

x = np.arange(len(class_names))
width = 0.35

fig, ax = plt.subplots(figsize=(14, 6))
bars1 = ax.bar(x - width/2, f1_cv, width, label='CV Model', alpha=0.8, color='#1f77b4')
bars2 = ax.bar(x + width/2, f1_mm, width, label='Multimodal', alpha=0.8, color='#2ca02c')

ax.set_xlabel('Diagnosis Type', fontsize=12, fontweight='bold')
ax.set_ylabel('F1-Score', fontsize=12, fontweight='bold')
ax.set_title('F1-Score Comparison by Class', fontsize=16, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_names, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 1])

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('f1_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# %%
# 4. CLASSIFICATION REPORTS
print("\n" + "="*70)
print("CV MODEL - CLASSIFICATION REPORT")
print("="*70)
print(classification_report(y_test, y_pred_cv, target_names=class_names, digits=4))

print("\n" + "="*70)
print("MULTIMODAL MODEL - CLASSIFICATION REPORT")
print("="*70)
print(classification_report(y_test, y_pred_mm, target_names=class_names, digits=4))

print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"CV Model Accuracy: {test_accuracy_cv:.4f}")
print(f"Multimodal Accuracy: {test_accuracy_mm:.4f}")
print(f"Improvement: {(test_accuracy_mm - test_accuracy_cv)*100:+.2f}%")

Generating predictions for both models...
47/47 [==============================] - 4s 78ms/step


/tmp/ipykernel_2248/1295861189.py:60: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_2248/1295861189.py:84: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



CV MODEL - CLASSIFICATION REPORT
              precision    recall  f1-score   support

       akiec     0.6111    0.6735    0.6408        49
         bcc     0.7714    0.7013    0.7347        77
         bkl     0.8029    0.6667    0.7285       165
          df     0.8182    0.5294    0.6429        17
         mel     0.6605    0.6407    0.6505       167
          nv     0.9089    0.9423    0.9253      1006
        vasc     0.7692    0.9091    0.8333        22

    accuracy                         0.8523      1503
   macro avg     0.7632    0.7233    0.7366      1503
weighted avg     0.8499    0.8523    0.8496      1503


MULTIMODAL MODEL - CLASSIFICATION REPORT
              precision    recall  f1-score   support

       akiec     0.5224    0.7143    0.6034        49
         bcc     0.6744    0.7532    0.7117        77
         bkl     0.5714    0.6545    0.6102       165
          df     0.6667    0.5882    0.6250        17
         mel     0.5149    0.6228    0.5637       167
  

/tmp/ipykernel_2248/1295861189.py:121: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [24]:
model_cv.save_weights('model_weights.h5')
model_cv.save('HAM_ResNet_final.keras')  
model_cv.save('HAM_ResNet_final.h5')     
model_cv.save('HAM_ResNet_final')  # This creates a directory
print("✓ Model saved in SavedModel format")
print("✓ Model saved!")
print("✓ Weights saved")


/home/francisco_ardoso/miniconda3/envs/tf-clean/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: HAM_ResNet_final/assets


INFO:tensorflow:Assets written to: HAM_ResNet_final/assets


✓ Model saved in SavedModel format
✓ Model saved!
✓ Weights saved
